# Squeeze-and-Excitation Networks
https://arxiv.org/pdf/1709.01507.pdf

ILSVRC2017優勝モデル<br>
**“Squeeze-and-Excitation”(SE) block**の提案<br>
チャンネル間の相互依存性をはっきりとモデリングすることで、チャンネル面での特徴を再構成


ResidualAttentionNetworkが**空間方向**へのAttentionだったのに対して、こちらは**チャンネル**へのAttention

<img src='img/seblock.png'>

## 1.構造

### - Phase1(普通のConv)
$F_{tr} : X → U,$　　($X ∈ R^{W'×H'×C'}$,$ U ∈ R^{W×H×C} $)

### - Phase2(Squeeze operation)
Uの特徴は次に** squeeze operation**に通される。<br>
Global Average Poolingをして1x1xCに

### - Phase3(Excitation operation)
FC -> RELU -> FC ->sigmoidでどのチャンネルに注目したらいいのかAttentionMask（と解釈）を生成

### - Phase4(Scale)
$U$に得られた1x1xCの重みをかける



# 2.SEBlockのkeras実装

<img src='img/senet_resnet.png' width=400>

<img src ='img/senet_results.png'>

In [1]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Conv2D
from keras.layers import add
from keras.layers import multiply
from keras import backend as K
from keras.layers.core import Flatten

Using TensorFlow backend.


In [2]:
def squeeze_excite_block(input, ratio=2):
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

# Simple Structure

In [3]:
inputs = Input(shape=(28,28,1))
x = Conv2D(32, (3, 3), padding='same', strides=1,activation='relu')(inputs)
x = squeeze_excite_block(x)
x = Conv2D(1, (1, 1), padding='same', strides=1,activation='relu')(x)
x = Flatten()(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(input=inputs, output=x)

/Users/tomoki/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`


In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 28, 28, 32)    320         input_1[0][0]                    
____________________________________________________________________________________________________
global_average_pooling2d_1 (Glob (None, 32)            0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
reshape_1 (Reshape)              (None, 1, 1, 32)      0           global_average_pooling2d_1[0][0] 
___________________________________________________________________________________________

# SE-Residual Block 

In [7]:
inputs = Input(shape=(28,28,1))
x = Conv2D(32, (3, 3), padding='same', strides=1,activation='relu')(inputs)

residual =x

x = squeeze_excite_block(x)
x = add([x,residual])

x = Conv2D(1, (1, 1), padding='same', strides=1,activation='relu')(x)

x = Flatten()(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(input=inputs, output=x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
conv2d_7 (Conv2D)                (None, 28, 28, 32)    320         input_4[0][0]                    
____________________________________________________________________________________________________
global_average_pooling2d_4 (Glob (None, 32)            0           conv2d_7[0][0]                   
____________________________________________________________________________________________________
reshape_4 (Reshape)              (None, 1, 1, 32)      0           global_average_pooling2d_4[0][0] 
___________________________________________________________________________________________

/Users/tomoki/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`


## Visualize the Activatio  of Excitation

<img src='img/SEchart.png'>

最初の方は違いがないが、層を重ねるごとに重視する層の差が顕著になってくる。